# Recruit Restaurant Visitor Forecasting Version III

Following:
https://github.com/dongzhang84/Kaggle/blob/master/Restaurant_Visitor_Forecasting/Visitor_forecasting_v2.ipynb


In [1]:
import glob, re
import numpy as np
import pandas as pd
from sklearn import *
from datetime import datetime
from xgboost import XGBRegressor

## Load Data

In [2]:
df_ar = pd.read_csv('data/air_reserve.csv')
df_hr = pd.read_csv('data/hpg_reserve.csv')
df_astore = pd.read_csv('data/air_store_info.csv')
df_hstore = pd.read_csv('data/hpg_store_info.csv')
df_storeid = pd.read_csv('data/store_id_relation.csv')
df_av = pd.read_csv('data/air_visit_data.csv')
date_info = pd.read_csv('data/date_info.csv')
df_submission = pd.read_csv('data/sample_submission.csv')

The data description see:

https://github.com/dongzhang84/Kaggle/blob/master/Restaurant_Visitor_Forecasting/Visitor_forecasting_v1.ipynb

In [3]:
df_ar.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


In [4]:
df_hr.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


In [5]:
df_astore.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


In [6]:
df_hstore.head()

,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_6622b62385aec8bf,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
1,hpg_e9e068dd49c5fa00,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
2,hpg_2976f7acb4b3a3bc,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
3,hpg_e51a522e098f024c,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
4,hpg_e3d0e1519894f275,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221


In [7]:
df_storeid.head()

,air_store_id,hpg_store_id
0,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a
1,air_a24bf50c3e90d583,hpg_c34b496d0305a809
2,air_c7f78b4f3cba33ff,hpg_cd8ae0d9bbd58ff9
3,air_947eb2cae4f3e8f2,hpg_de24ea49dc25d6b8
4,air_965b2e0cf4119003,hpg_653238a84804d8e7


In [8]:
df_av.head()

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


In [9]:
date_info.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [10]:
df_submission.head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0


## TODO next

- df_av, dayofweek, year, month
- df_submission, do the same thing
- stores, df_av-like dataframe. Do min, max, mean, median for df_av and merge with stores
- df_hr, merge with df_storeid, do reserve_lag, reserve_visitor_mean
- df_ar, also do reserve_lag, reserve_visitor_mean
- merge df_hr and df_ar with df_av, and mean these two
- add restaurant information

- Similar steps for df_submission

In [11]:
df_av.shape

(252108, 3)

In [12]:
df_av.dtypes

air_store_id    object
visit_date      object
visitors         int64
dtype: object

In [13]:
df_av['visit_date'] = pd.to_datetime(df_av.visit_date)

In [14]:
df_av['dow'] = df_av.visit_date.dt.dayofweek
df_av.head(3)

,air_store_id,visit_date,visitors,dow
0,air_ba937bf13d40fb24,2016-01-13,25,2
1,air_ba937bf13d40fb24,2016-01-14,32,3
2,air_ba937bf13d40fb24,2016-01-15,29,4


In [15]:
df_av['year'] = df_av.visit_date.dt.year
df_av['month'] = df_av.visit_date.dt.month

In [16]:
df_av['date'] = df_av.visit_date.dt.day

In [17]:
df_av.head(3)

,air_store_id,visit_date,visitors,dow,year,month,date
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1,13
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1,14
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1,15


In [35]:
#(df_av.visit_date - pd.to_datetime('2016-01-13')).dt.days

In [18]:
df_submission.head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0


In [19]:
df_submission['air_store_id'] = df_submission.id.apply(lambda x: x[:-11])

In [20]:
df_submission['visit_date']  = df_submission.id.apply(lambda x: x[-10:])
df_submission['visit_date'] = pd.to_datetime(df_submission.visit_date)

In [21]:
df_submission['dow'] = df_submission.visit_date.dt.dayofweek
df_submission['year'] = df_submission.visit_date.dt.year
df_submission['month'] = df_submission.visit_date.dt.month
df_submission['date'] = df_submission.visit_date.dt.day

In [22]:
df_submission.head()

,id,visitors,air_store_id,visit_date,dow,year,month,date
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23,6,2017,4,23
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24,0,2017,4,24
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25,1,2017,4,25
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26,2,2017,4,26
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27,3,2017,4,27


In [42]:
# Next to create: stores

unique_stores = df_submission['air_store_id'].unique()
stores = pd.concat([pd.DataFrame({'air_store_id': unique_stores, 'dow': [i]*len(unique_stores)}) for i in range(7)], 
                   axis=0, ignore_index=True).reset_index(drop=True)

In [43]:
stores

,air_store_id,dow
0,air_00a91d42b08b08d9,0
1,air_0164b9927d20bcc3,0
2,air_0241aa3964b7f861,0
3,air_0328696196e46f18,0
4,air_034a3d5b40d5b1b1,0
5,air_036d4f1ee7285390,0
6,air_0382c794b73b51ad,0
7,air_03963426c9312048,0
8,air_04341b588bde96cd,0
9,air_049f6d5b402a31b2,0


In [25]:
#Do min, max, mean, median for df_av and merge with stores

df_av.head()

,air_store_id,visit_date,visitors,dow,year,month,date
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1,13
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1,14
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1,15
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1,16
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1,18


In [40]:
# max

temp1 = df_av.groupby(['air_store_id','dow'])['visitors'].max().reset_index()
temp1 = temp1.rename(columns={'visitors':'max_visitors'})
temp1.head()

,air_store_id,dow,max_visitors
0,air_00a91d42b08b08d9,0,47
1,air_00a91d42b08b08d9,1,43
2,air_00a91d42b08b08d9,2,52
3,air_00a91d42b08b08d9,3,47
4,air_00a91d42b08b08d9,4,57


In [34]:
temp1.shape

(5741, 3)

In [44]:
stores = stores.merge(temp1, on=['air_store_id','dow'], how='left')
stores

,air_store_id,dow,max_visitors
0,air_00a91d42b08b08d9,0,47.0
1,air_0164b9927d20bcc3,0,19.0
2,air_0241aa3964b7f861,0,23.0
3,air_0328696196e46f18,0,27.0
4,air_034a3d5b40d5b1b1,0,66.0
5,air_036d4f1ee7285390,0,38.0
6,air_0382c794b73b51ad,0,47.0
7,air_03963426c9312048,0,70.0
8,air_04341b588bde96cd,0,76.0
9,air_049f6d5b402a31b2,0,20.0


In [39]:
stores.dtypes

air_store_id     object
dow               int64
max_visitors    float64
dtype: object

In [45]:
# min

temp1 = df_av.groupby(['air_store_id','dow'])['visitors'].min().reset_index()
temp1 = temp1.rename(columns={'visitors':'min_visitors'})
temp1.head()

,air_store_id,dow,min_visitors
0,air_00a91d42b08b08d9,0,1
1,air_00a91d42b08b08d9,1,1
2,air_00a91d42b08b08d9,2,15
3,air_00a91d42b08b08d9,3,15
4,air_00a91d42b08b08d9,4,17


In [46]:
stores = stores.merge(temp1, on=['air_store_id','dow'], how='left')
stores

,air_store_id,dow,max_visitors,min_visitors
0,air_00a91d42b08b08d9,0,47.0,1.0
1,air_0164b9927d20bcc3,0,19.0,2.0
2,air_0241aa3964b7f861,0,23.0,2.0
3,air_0328696196e46f18,0,27.0,2.0
4,air_034a3d5b40d5b1b1,0,66.0,1.0
5,air_036d4f1ee7285390,0,38.0,4.0
6,air_0382c794b73b51ad,0,47.0,1.0
7,air_03963426c9312048,0,70.0,2.0
8,air_04341b588bde96cd,0,76.0,5.0
9,air_049f6d5b402a31b2,0,20.0,2.0


In [49]:
# median

tmp = df_av.groupby(['air_store_id','dow'], as_index=False)['visitors']\
      .median().rename(columns={'visitors':'median_visitors'})

In [51]:
temp1 = df_av.groupby(['air_store_id','dow'])['visitors'].median().reset_index()
temp1 = temp1.rename(columns={'visitors':'median_visitors'})
print(temp1.shape)
temp1.head()

(5741, 3)


,air_store_id,dow,median_visitors
0,air_00a91d42b08b08d9,0,19.0
1,air_00a91d42b08b08d9,1,24.5
2,air_00a91d42b08b08d9,2,28.0
3,air_00a91d42b08b08d9,3,30.0
4,air_00a91d42b08b08d9,4,35.5


In [52]:
stores = stores.merge(temp1, on=['air_store_id','dow'], how='left')
stores.head()

,air_store_id,dow,max_visitors,min_visitors,median_visitors
0,air_00a91d42b08b08d9,0,47.0,1.0,19.0
1,air_0164b9927d20bcc3,0,19.0,2.0,6.0
2,air_0241aa3964b7f861,0,23.0,2.0,8.0
3,air_0328696196e46f18,0,27.0,2.0,4.0
4,air_034a3d5b40d5b1b1,0,66.0,1.0,10.0


In [53]:
# mean

temp1 = df_av.groupby(['air_store_id','dow'])['visitors'].mean().reset_index()
temp1 = temp1.rename(columns={'visitors':'mean_visitors'})
print(temp1.shape)

stores = stores.merge(temp1, on=['air_store_id','dow'], how='left')
stores.head()

(5741, 3)


,air_store_id,dow,max_visitors,min_visitors,median_visitors,mean_visitors
0,air_00a91d42b08b08d9,0,47.0,1.0,19.0,22.457143
1,air_0164b9927d20bcc3,0,19.0,2.0,6.0,7.500000
2,air_0241aa3964b7f861,0,23.0,2.0,8.0,8.920635
3,air_0328696196e46f18,0,27.0,2.0,4.0,6.416667
4,air_034a3d5b40d5b1b1,0,66.0,1.0,10.0,11.864865


In [54]:
# count

temp1 = df_av.groupby(['air_store_id','dow'])['visitors'].count().reset_index()
temp1 = temp1.rename(columns={'visitors':'count_visitors'})
print(temp1.shape)

stores = stores.merge(temp1, on=['air_store_id','dow'], how='left')
stores.head()

(5741, 3)


,air_store_id,dow,max_visitors,min_visitors,median_visitors,mean_visitors,count_visitors
0,air_00a91d42b08b08d9,0,47.0,1.0,19.0,22.457143,35.0
1,air_0164b9927d20bcc3,0,19.0,2.0,6.0,7.500000,20.0
2,air_0241aa3964b7f861,0,23.0,2.0,8.0,8.920635,63.0
3,air_0328696196e46f18,0,27.0,2.0,4.0,6.416667,12.0
4,air_034a3d5b40d5b1b1,0,66.0,1.0,10.0,11.864865,37.0


### df_hr, merge with df_storeid, do reserve_lag, reserve_visitor_mean

In [76]:
df_hr = pd.read_csv('data/hpg_reserve.csv')
df_hr.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


In [56]:
df_storeid.head()

,air_store_id,hpg_store_id
0,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a
1,air_a24bf50c3e90d583,hpg_c34b496d0305a809
2,air_c7f78b4f3cba33ff,hpg_cd8ae0d9bbd58ff9
3,air_947eb2cae4f3e8f2,hpg_de24ea49dc25d6b8
4,air_965b2e0cf4119003,hpg_653238a84804d8e7


In [77]:
df_hr = pd.merge(df_hr, df_storeid, how='inner', on='hpg_store_id')
df_hr.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,air_store_id
0,hpg_878cc70b1abc76f7,2016-01-01 19:00:00,2016-01-01 15:00:00,4,air_db80363d35f10926
1,hpg_878cc70b1abc76f7,2016-01-02 19:00:00,2016-01-02 14:00:00,2,air_db80363d35f10926
2,hpg_878cc70b1abc76f7,2016-01-03 18:00:00,2016-01-02 20:00:00,6,air_db80363d35f10926
3,hpg_878cc70b1abc76f7,2016-01-06 20:00:00,2016-01-04 22:00:00,3,air_db80363d35f10926
4,hpg_878cc70b1abc76f7,2016-01-11 18:00:00,2016-01-11 14:00:00,2,air_db80363d35f10926


In [78]:
df_hr['visit_datetime'] = pd.to_datetime(df_hr['visit_datetime'])

In [79]:
df_hr['visit_datetime'] = df_hr['visit_datetime'].dt.date

In [80]:
df_hr['reserve_datetime'] = pd.to_datetime(df_hr['reserve_datetime'])
df_hr['reserve_datetime'] = df_hr['reserve_datetime'].dt.date

In [81]:
df_hr['reserve_datetime_diff'] = (df_hr['visit_datetime'] - df_hr['reserve_datetime']).dt.days

In [82]:
df_hr.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,air_store_id,reserve_datetime_diff
0,hpg_878cc70b1abc76f7,2016-01-01,2016-01-01,4,air_db80363d35f10926,0
1,hpg_878cc70b1abc76f7,2016-01-02,2016-01-02,2,air_db80363d35f10926,0
2,hpg_878cc70b1abc76f7,2016-01-03,2016-01-02,6,air_db80363d35f10926,1
3,hpg_878cc70b1abc76f7,2016-01-06,2016-01-04,3,air_db80363d35f10926,2
4,hpg_878cc70b1abc76f7,2016-01-11,2016-01-11,2,air_db80363d35f10926,0


In [83]:
tmp1 = df_hr.groupby(['air_store_id','visit_datetime'], as_index=False)\
       [['reserve_datetime_diff', 'reserve_visitors']].sum()\
       .rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs1', 'reserve_visitors':'rv1'})

In [84]:
tmp2 = df_hr.groupby(['air_store_id','visit_datetime'], as_index=False)\
       [['reserve_datetime_diff', 'reserve_visitors']].mean()\
       .rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs2', 'reserve_visitors':'rv2'})

In [85]:
df_hr = pd.merge(tmp1, tmp2, how='inner', on=['air_store_id','visit_date'])
df_hr.head()

,air_store_id,visit_date,rs1,rv1,rs2,rv2
0,air_00a91d42b08b08d9,2016-01-14,3,2,3.0,2.0
1,air_00a91d42b08b08d9,2016-01-15,6,4,6.0,4.0
2,air_00a91d42b08b08d9,2016-01-16,3,2,3.0,2.0
3,air_00a91d42b08b08d9,2016-01-22,3,2,3.0,2.0
4,air_00a91d42b08b08d9,2016-01-29,6,5,6.0,5.0


### df_ar, merge with df_storeid, do reserve_lag, reserve_visitor_mean

In [86]:
df_ar['visit_datetime'] = pd.to_datetime(df_ar['visit_datetime'])
df_ar['visit_datetime'] = df_ar['visit_datetime'].dt.date

df_ar['reserve_datetime'] = pd.to_datetime(df_ar['reserve_datetime'])
df_ar['reserve_datetime'] = df_ar['reserve_datetime'].dt.date

In [87]:
df_ar['reserve_datetime_diff'] = (df_ar['visit_datetime'] - df_ar['reserve_datetime']).dt.days

In [88]:
tmp1 = df_ar.groupby(['air_store_id','visit_datetime'], as_index=False)\
       [['reserve_datetime_diff', 'reserve_visitors']].sum()\
       .rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs1', 'reserve_visitors':'rv1'})

In [89]:
tmp2 = df_ar.groupby(['air_store_id','visit_datetime'], as_index=False)\
       [['reserve_datetime_diff', 'reserve_visitors']].mean()\
       .rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs2', 'reserve_visitors':'rv2'})

In [90]:
df_ar = pd.merge(tmp1, tmp2, how='inner', on=['air_store_id','visit_date'])
df_ar.head()

,air_store_id,visit_date,rs1,rv1,rs2,rv2
0,air_00a91d42b08b08d9,2016-10-31,0,2,0.0,2.0
1,air_00a91d42b08b08d9,2016-12-05,4,9,4.0,9.0
2,air_00a91d42b08b08d9,2016-12-14,6,18,6.0,18.0
3,air_00a91d42b08b08d9,2016-12-17,6,2,6.0,2.0
4,air_00a91d42b08b08d9,2016-12-20,2,4,2.0,4.0


### merge df_hr and df_ar with df_av, and create df_train

In [91]:
df_av.shape

(252108, 7)

In [92]:
df_ar.shape

(29830, 6)

In [101]:
df_ar.head()

,air_store_id,visit_date,rs1,rv1,rs2,rv2
0,air_00a91d42b08b08d9,2016-10-31,0,2,0.0,2.0
1,air_00a91d42b08b08d9,2016-12-05,4,9,4.0,9.0
2,air_00a91d42b08b08d9,2016-12-14,6,18,6.0,18.0
3,air_00a91d42b08b08d9,2016-12-17,6,2,6.0,2.0
4,air_00a91d42b08b08d9,2016-12-20,2,4,2.0,4.0


In [100]:
df_ar['visit_date'] = pd.to_datetime(df_ar['visit_date'])

In [108]:
df_train = df_av.merge(df_ar, how = 'left', on=['air_store_id','visit_date'])
df_train.shape

(252108, 11)

In [110]:
df_train.head()

,air_store_id,visit_date,visitors,dow,year,month,date,rs1,rv1,rs2,rv2
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1,13,NaN,NaN,NaN,NaN
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1,14,NaN,NaN,NaN,NaN
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1,15,NaN,NaN,NaN,NaN
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1,16,NaN,NaN,NaN,NaN
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1,18,NaN,NaN,NaN,NaN


In [109]:
df_train.isnull().sum()

air_store_id         0
visit_date           0
visitors             0
dow                  0
year                 0
month                0
date                 0
rs1             224044
rv1             224044
rs2             224044
rv2             224044
dtype: int64

In [112]:
df_hr['visit_date'] = pd.to_datetime(df_hr['visit_date'])
df_train = df_train.merge(df_hr, how = 'left', on=['air_store_id','visit_date'])

In [113]:
df_train.shape

(252108, 15)

In [115]:
df_train.isnull().sum()

air_store_id         0
visit_date           0
visitors             0
dow                  0
year                 0
month                0
date                 0
rs1_x           224044
rv1_x           224044
rs2_x           224044
rv2_x           224044
rs1_y           238558
rv1_y           238558
rs2_y           238558
rv2_y           238558
dtype: int64

### merge df_hr and df_ar with df_submission, and create df_test

In [107]:
df_submission.head()

,id,visitors,air_store_id,visit_date,dow,year,month,date
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23,6,2017,4,23
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24,0,2017,4,24
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25,1,2017,4,25
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26,2,2017,4,26
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27,3,2017,4,27


In [116]:
df_test = df_submission[['air_store_id','visit_date','visitors','dow','year','month','date']]
df_test.head()

,air_store_id,visit_date,visitors,dow,year,month,date
0,air_00a91d42b08b08d9,2017-04-23,0,6,2017,4,23
1,air_00a91d42b08b08d9,2017-04-24,0,0,2017,4,24
2,air_00a91d42b08b08d9,2017-04-25,0,1,2017,4,25
3,air_00a91d42b08b08d9,2017-04-26,0,2,2017,4,26
4,air_00a91d42b08b08d9,2017-04-27,0,3,2017,4,27


In [117]:
df_test.dtypes

air_store_id            object
visit_date      datetime64[ns]
visitors                 int64
dow                      int64
year                     int64
month                    int64
date                     int64
dtype: object

In [118]:
df_test = df_test.merge(df_ar, how = 'left', on=['air_store_id','visit_date'])
df_test = df_test.merge(df_hr, how = 'left', on=['air_store_id','visit_date'])

df_test.head()

,air_store_id,visit_date,visitors,dow,year,month,date,rs1_x,rv1_x,rs2_x,rv2_x,rs1_y,rv1_y,rs2_y,rv2_y
0,air_00a91d42b08b08d9,2017-04-23,0,6,2017,4,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,air_00a91d42b08b08d9,2017-04-24,0,0,2017,4,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,air_00a91d42b08b08d9,2017-04-25,0,1,2017,4,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,air_00a91d42b08b08d9,2017-04-26,0,2,2017,4,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,air_00a91d42b08b08d9,2017-04-27,0,3,2017,4,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## add restaurant information to stores

In [120]:
stores.head()

,air_store_id,dow,max_visitors,min_visitors,median_visitors,mean_visitors,count_visitors
0,air_00a91d42b08b08d9,0,47.0,1.0,19.0,22.457143,35.0
1,air_0164b9927d20bcc3,0,19.0,2.0,6.0,7.500000,20.0
2,air_0241aa3964b7f861,0,23.0,2.0,8.0,8.920635,63.0
3,air_0328696196e46f18,0,27.0,2.0,4.0,6.416667,12.0
4,air_034a3d5b40d5b1b1,0,66.0,1.0,10.0,11.864865,37.0
